# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,13.79,91,100,2.68,NZ,1702330274
1,1,georgetown,5.4112,100.3354,26.08,97,20,1.54,MY,1702330697
2,2,margaret river,-33.9500,115.0667,15.59,72,0,4.31,AU,1702330270
3,3,kaa-khem,51.6833,94.7333,-34.22,97,89,1.00,RU,1702330708
4,4,albany,42.6001,-73.9662,2.98,74,12,5.81,US,1702330212


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
city_data_df.hvplot.scatter("Lng", "Lat", xlabel="x", ylabel="y", by="City", size="Humidity")

:NdOverlay   [City]
   :Scatter   [Lng]   (Lat,Humidity)

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) 
                                 & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,73,ovalle,-30.5983,-71.2003,26.01,38,0,2.46,CL,1702330714
201,201,banfora,10.6333,-4.7667,25.24,14,0,1.63,BF,1702330728
205,205,karratha,-20.7377,116.8463,24.52,49,0,2.70,AU,1702330728
231,231,isabela,6.7085,121.9711,23.03,80,0,2.35,PH,1702330732
294,294,chiang mai,18.7500,99.0000,23.04,94,0,0.51,TH,1702330739
307,307,geraldton,-28.7667,114.6000,22.23,28,0,4.12,AU,1702330739
333,333,nouna,12.7333,-3.8667,24.49,15,0,3.17,BF,1702330744
418,418,carnarvon,-24.8667,113.6333,24.43,39,0,2.78,AU,1702330759
438,438,bougouni,11.4170,-7.4790,24.06,15,0,2.23,ML,1702330763
527,527,vallenar,-28.5708,-70.7581,26.56,33,0,3.21,CL,1702330778


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
73,ovalle,CL,-30.5983,-71.2003,38,None
201,banfora,BF,10.6333,-4.7667,14,None
205,karratha,AU,-20.7377,116.8463,49,None
231,isabela,PH,6.7085,121.9711,80,None
294,chiang mai,TH,18.7500,99.0000,94,None
307,geraldton,AU,-28.7667,114.6000,28,None
333,nouna,BF,12.7333,-3.8667,15,None
418,carnarvon,AU,-24.8667,113.6333,39,None
438,bougouni,ML,11.4170,-7.4790,15,None
527,vallenar,CL,-28.5708,-70.7581,33,None


In [12]:
#hotel_df[73]["Lng"]
hotel_df.set_index((1, len(hotel_df)))
print(hotel_df.get(3))

KeyError: 'None of [(1, 10)] are in the columns'

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
params = {
    "filter" :filters,
    #"limit" : limit,
    "bias" : bias,
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df[row]["Lat"]# YOUR CODE HERE
    lng = hotel_df[row]["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = hotel_df[]# YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params) # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)